In [5]:
a = ["rider_last14d_avg_driver_to_restaurant_time",
"rider_last7d_avg_driver_to_restaurant_time",
"rider_last3d_avg_driver_to_restaurant_time",
"rider_last1d_avg_driver_to_restaurant_time",
"rider_last14d_weekday_avg_driver_to_restaurant_time",
"rider_last7d_weekday_avg_driver_to_restaurant_time",
"rider_last3d_weekday_avg_driver_to_restaurant_time",
"rider_last1d_weekday_avg_driver_to_restaurant_time",
"rider_last28d_weekend_avg_driver_to_restaurant_time",
"rider_last21d_weekend_avg_driver_to_restaurant_time",
"rider_last14d_weekend_avg_driver_to_restaurant_time",
"rider_last7d_weekend_avg_driver_to_restaurant_time",
"rider_last3d_weekend_avg_driver_to_restaurant_time",
"rider_last1d_weekend_avg_driver_to_restaurant_time",
"rtt_last14d_weekday_avg_uwt",
"rtt_last7d_weekday_avg_uwt",
"rtt_last3d_weekday_avg_uwt",
"rtt_last1d_weekday_avg_uwt",
"rtt_last28d_weekend_avg_uwt",
"rtt_last21d_weekend_avg_uwt",
"rtt_last14d_weekend_avg_uwt",
"rtt_last7d_weekend_avg_uwt",
"rtt_last3d_weekend_avg_uwt",
"rtt_last1d_weekend_avg_uwt",
"rtt_last14d_avg_cooking_time",
"rtt_last7d_avg_cooking_time",
"rtt_last3d_avg_cooking_time",
"rtt_last1d_avg_cooking_time",
"rtt_last14d_weekday_avg_cooking_time",
"rtt_last7d_weekday_avg_cooking_time",
"rtt_last3d_weekday_avg_cooking_time",
"rtt_last1d_weekday_avg_cooking_time",
"rtt_last28d_weekend_avg_cooking_time",
"rtt_last21d_weekend_avg_cooking_time",
"rtt_last14d_weekend_avg_cooking_time",
"rtt_last7d_weekend_avg_cooking_time",
"rtt_last3d_weekend_avg_cooking_time",
"rtt_last1d_weekend_avg_cooking_time",
"rtt_last14d_avg_leaving_time",
"rtt_last7d_avg_leaving_time",
"rtt_last3d_avg_leaving_time",
"rtt_last1d_avg_leaving_time",
"rtt_last14d_weekday_avg_leaving_time",
"rtt_last7d_weekday_avg_leaving_time",
"rtt_last3d_weekday_avg_leaving_time",
"rtt_last1d_weekday_avg_leaving_time",
"rtt_last28d_weekend_avg_leaving_time",
"rtt_last21d_weekend_avg_leaving_time",
"rtt_last14d_weekend_avg_leaving_time",
"rtt_last7d_weekend_avg_leaving_time",
"rtt_last3d_weekend_avg_leaving_time",
"rtt_last1d_weekend_avg_leaving_time",
"rtt_last14d_avg_distance",
"rtt_last7d_avg_distance",
"rtt_last3d_avg_distance",
"rtt_last1d_avg_distance",
"rtt_last14d_weekday_avg_distance",
"rtt_last7d_weekday_avg_distance",
"rtt_last3d_weekday_avg_distance",
"rtt_last1d_weekday_avg_distance",
"rtt_last28d_weekend_avg_distance",
"rtt_last21d_weekend_avg_distance",
"rtt_last14d_weekend_avg_distance",
"rtt_last7d_weekend_avg_distance",
"rtt_last3d_weekend_avg_distance",
"rtt_last1d_weekend_avg_distance"
]

In [3]:
"rtt_last14d_avg_distance".replace("14d",'7d')

'rtt_last7d_avg_distance'

In [7]:
res = ""
for t in a :
    if t.find('7d')!=-1:
        res+="coalesce({},{}) as {},".format(t,t.replace('7d','14d'),t)
    elif t.find('3d')!=-1:
        res+="coalesce({},{},{}) as {},".format(t,t.replace('3d','7d'),t.replace('3d','14d'),t)
    elif t.find('t1d')!=-1:
        res+="coalesce({},{},{},{}) as {},".format(t,t.replace('1d','3d'),t.replace('1d','7d'),t.replace('1d','14d'),t)
    else:
        res+='{},'.format(t)
print(res)

rider_last14d_avg_driver_to_restaurant_time,coalesce(rider_last7d_avg_driver_to_restaurant_time,rider_last14d_avg_driver_to_restaurant_time) as rider_last7d_avg_driver_to_restaurant_time,coalesce(rider_last3d_avg_driver_to_restaurant_time,rider_last7d_avg_driver_to_restaurant_time,rider_last14d_avg_driver_to_restaurant_time) as rider_last3d_avg_driver_to_restaurant_time,coalesce(rider_last1d_avg_driver_to_restaurant_time,rider_last3d_avg_driver_to_restaurant_time,rider_last7d_avg_driver_to_restaurant_time,rider_last14d_avg_driver_to_restaurant_time) as rider_last1d_avg_driver_to_restaurant_time,rider_last14d_weekday_avg_driver_to_restaurant_time,coalesce(rider_last7d_weekday_avg_driver_to_restaurant_time,rider_last14d_weekday_avg_driver_to_restaurant_time) as rider_last7d_weekday_avg_driver_to_restaurant_time,coalesce(rider_last3d_weekday_avg_driver_to_restaurant_time,rider_last7d_weekday_avg_driver_to_restaurant_time,rider_last14d_weekday_avg_driver_to_restaurant_time) as rider_last3d

with rtt_h3_month as (
    select
        order_id,
        uwt,
        rtt_id,
        fp_h3_hash(from_location_lat, from_location_lng, 7) as rtt_h3_7,
        fp_h3_hash(from_location_lat, from_location_lng, 8) as rtt_h3_8,
        fp_h3_hash(from_location_lat, from_location_lng, 6) as rtt_h3_6,

        -- 1.4km 
        fp_h3_hash(to_location_lat, to_location_lng, 8) as user_h3_8,
        HOUR(created_at) as hour,
        CASE
            WHEN DAYOFWEEK(created_at) = 1
            OR DAYOFWEEK(created_at) = 7 THEN 1
            ELSE 0
        END AS is_weekend,
        floor(cast(created_at as int) / 60) * 60 as window_end_epoch,
        --时间窗
        floor(cast(driver_times_assigning_driver as int) / 5) * 5 as dispatch_window_epoch,
        cast(driver_times_arrived_from as int) - cast(created_at as int) as driver_to_restaurant,
        dt
    from
        feature_beta.dalian_sds_order_total_daily_v3
    where
        isNotNull(driver_times_completed)
        and (delivery_status = 'COMPLETED')
        and (status = 'COMPLETED')
        and (delivery_supplier = 'lineman_driver')
        and floor(
            cast(driver_times_completed as double) - cast(user_placed_time as double)
        ) > (5 * 60)
        and floor(
            cast(driver_times_completed as double) - cast(user_placed_time as double)
        ) < (90 * 60)
        and distance >= 200
        and dt between replace(date_sub('$start_date_dash', 28), '-', '')
        and replace('$start_date_dash', '-', '')
)
select rtt_h3_id as h3hash ,   '$partition' as dt
,rider_last14d_avg_driver_to_restaurant_time,
rider_last7d_avg_driver_to_restaurant_time,
rider_last3d_avg_driver_to_restaurant_time,
rider_last1d_avg_driver_to_restaurant_time,-- 历史N天， 周末/周常 ，avg到店时间
rider_last14d_weekday_avg_driver_to_restaurant_time,
rider_last7d_weekday_avg_driver_to_restaurant_time,
rider_last3d_weekday_avg_driver_to_restaurant_time,
rider_last1d_weekday_avg_driver_to_restaurant_time,
rider_last28d_weekend_avg_driver_to_restaurant_time,
rider_last21d_weekend_avg_driver_to_restaurant_time,
rider_last14d_weekend_avg_driver_to_restaurant_time,
rider_last7d_weekend_avg_driver_to_restaurant_time,
rider_last3d_weekend_avg_driver_to_restaurant_time,
rider_last1d_weekend_avg_driver_to_restaurant_time,-- 历史N天,周末/周常 平均UWT
rtt_last14d_weekday_avg_uwt,
rtt_last7d_weekday_avg_uwt,
rtt_last3d_weekday_avg_uwt,
rtt_last1d_weekday_avg_uwt,
rtt_last28d_weekend_avg_uwt,
rtt_last21d_weekend_avg_uwt,
rtt_last14d_weekend_avg_uwt,
rtt_last7d_weekend_avg_uwt,
rtt_last3d_weekend_avg_uwt,
rtt_last1d_weekend_avg_uwt from (
select
    rtt_h3_7 as rtt_h3_id,
    -- 历史N 天 Z 到店时间 
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last14d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last7d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last3d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last1d_avg_driver_to_restaurant_time,
    -- 历史N天， 周末/周常 ，avg到店时间
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last14d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last7d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last3d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last1d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last28d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last21d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last14d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last7d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '')and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last3d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last1d_weekend_avg_driver_to_restaurant_time,
    -- 历史N天,周末/周常 平均UWT
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last14d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last7d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last3d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last1d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last28d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last21d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last14d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last7d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last3d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last1d_weekend_avg_uwt
from rtt_h3_month
group by rtt_h3_7 
union all 
select
    rtt_h3_8 as rtt_h3_id,
    -- 历史N 天 Z 到店时间 
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last14d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last7d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last3d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last1d_avg_driver_to_restaurant_time,
    -- 历史N天， 周末/周常 ，avg到店时间
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last14d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last7d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last3d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last1d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last28d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last21d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last14d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last7d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '')and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last3d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last1d_weekend_avg_driver_to_restaurant_time,
    -- 历史N天,周末/周常 平均UWT
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last14d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last7d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last3d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last1d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last28d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last21d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last14d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last7d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last3d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last1d_weekend_avg_uwt
from rtt_h3_month
group by rtt_h3_8 
union all
select
    rtt_h3_6 as rtt_h3_id,
    -- 历史N 天 Z 到店时间 
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last14d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last7d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last3d_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)) as rider_last1d_avg_driver_to_restaurant_time,
    -- 历史N天， 周末/周常 ，avg到店时间
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last14d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last7d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last3d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=0) as rider_last1d_weekday_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last28d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last21d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last14d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant) and is_weekend=1) as rider_last7d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '')and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last3d_weekend_avg_driver_to_restaurant_time,
    avg(driver_to_restaurant) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and isNotNull(driver_to_restaurant)  and is_weekend=1) as rider_last1d_weekend_avg_driver_to_restaurant_time,
    -- 历史N天,周末/周常 平均UWT
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last14d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last7d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last3d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=0) as rtt_last1d_weekday_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 28), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last28d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 21), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last21d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 14), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last14d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 7), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last7d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 3), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last3d_weekend_avg_uwt,
    avg(uwt) filter(where dt between replace(date_sub('$start_date_dash', 1), '-', '') and replace('$start_date_dash', '-', '') and is_weekend=1) as rtt_last1d_weekend_avg_uwt
from rtt_h3_month
group by rtt_h3_6
)

In [24]:
base = './base.sql'
exp = './exp.sql'



In [25]:
file = open(base)
line0 = ''
for line in file.readlines():
    lines = line.split("],")
    dicts = {}
    for ids in lines:
        key,value = ids.split(":")
        if key !='riders':
            value = value+']'
        dicts[key] = sorted((ast.literal_eval(value)))
        



In [48]:
a = line0.split("],")
a

["orders:['LMF-240510-964690022', 'LMF-240510-003224698'",
 'mp_id:[1, 2',
 "riders:['LMDBR2TRM', 'LMD9N00HH', 'LMDR1GL2G', 'LMDEHVTXX', 'LMDLKDEVW', 'LMDZAW4C2', 'LMDQIRS1Q', 'LMDWZNP0E', 'LMD7GJE9G', 'LMD8IRFJT', 'LMDUONDV1', 'LMDI2MHW6', 'LMDNHF3DB', 'LMD4OPY82', 'LMDGZSGBZ', 'LMDB3DN8R', 'LMDS4O9C1']\n"]

In [54]:
import ast
keys = []
values = []



In [55]:
dicts

{'orders': ['LMF-240510-003224698', 'LMF-240510-964690022'],
 'mp_id': [1, 2],
 'riders': ['LMD4OPY82',
  'LMD7GJE9G',
  'LMD8IRFJT',
  'LMD9N00HH',
  'LMDB3DN8R',
  'LMDBR2TRM',
  'LMDEHVTXX',
  'LMDGZSGBZ',
  'LMDI2MHW6',
  'LMDLKDEVW',
  'LMDNHF3DB',
  'LMDQIRS1Q',
  'LMDR1GL2G',
  'LMDS4O9C1',
  'LMDUONDV1',
  'LMDWZNP0E',
  'LMDZAW4C2']}

In [42]:
import json


TypeError: the JSON object must be str, bytes or bytearray, not dict